In [1]:
import retinanalysis as ra
import numpy as np

/Users/racheloaks-leaf/anaconda3/envs/analysis2/lib/python3.11/site-packages/datajoint/plugin.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-07-29 12:18:19,245][INFO]: DataJoint 0.14.4 connected to root@127.0.0.1:3306


FileNotFoundError: No config file found. Use reset_config() and create_config() to make one.

## Step 1: Query protocol across all experiments

In [ ]:
protocols_to_query = ['movingletters', 'presentmatfiles']
experiment_query = ra.get_datasets_from_protocol_names(protocols_to_query)
display(experiment_query)

## Select experiment and data file of interest

In [ ]:
exp_name = experiment_query.loc[2, 'exp_name']
datafile_name = experiment_query.loc[2,'datafile_name']
ls_param_names = ['magnificationFactor', 'matFile', 'imageOrder']
pipeline=ra.create_mea_pipeline(exp_name, datafile_name, ls_params = ls_param_names)
stim = pipeline.stim_block
response = pipeline.response_block
analysis = pipeline.analysis_chunk

In [ ]:
cell_types = ['OnP', 'OffP', 'OnM', 'OffM', 'SBC', 'Xmas']

roi_dict = {'x_min' : 700, 'x_max' : 1500, 'y_min' : 500, 'y_max' : 600}

rf_axes = pipeline.plot_rfs(cell_types = cell_types, units = 'pixels', roi = roi_dict)
timecourse_axes = pipeline.plot_timecourses(cell_types = cell_types, roi = roi_dict, roi_units = 'pixels');

In [ ]:
epoch_params = ['imageOrder']
image_order = [lst for lst in stim.df_epochs['imageOrder']]
image_order = np.array(image_order)

images_per_epoch = stim.df_epochs.loc[0,'epoch_parameters']['imagesPerEpoch']
num_epochs = len(stim.df_epochs)